In [1]:
import os
import librosa
import numpy as np
import soundfile as sf
from tqdm import tqdm

In [2]:
# Folder input dan output
input_dir = 'processed_dataset'
output_dir = 'augmented_dataset'
os.makedirs(output_dir, exist_ok=True)

In [3]:
# Target minimum per kelas
TARGET = 500

In [4]:
# --- Fungsi Augmentasi ---
def apply_pitch_shift(data, sr, n_steps=2):
    return librosa.effects.pitch_shift(data, sr=sr, n_steps=n_steps)

def apply_time_stretch(data, rate=1.2):
    return librosa.effects.time_stretch(data, rate)

def apply_add_noise(data, noise_factor=0.005):
    noise = np.random.randn(len(data))
    return data + noise_factor * noise

def apply_shift(data, shift_max=0.2):
    shift = int(np.random.uniform(-shift_max, shift_max) * len(data))
    return np.roll(data, shift)

def apply_volume_control(data, gain_db=5):
    return data * (10 ** (gain_db / 20))


In [5]:
# --- Proses Augmentasi ---
for label in os.listdir(input_dir):
    label_input_path = os.path.join(input_dir, label)
    label_output_path = os.path.join(output_dir, label)

    if not os.path.isdir(label_input_path):
        continue

    os.makedirs(label_output_path, exist_ok=True)

    # Ambil file .wav yang sudah ada di output
    existing_files = [f for f in os.listdir(label_output_path) if f.endswith(".wav")]
    current_count = len(existing_files)

    # Ambil file input dari folder asal
    input_files = [f for f in os.listdir(label_input_path) if f.endswith(".wav")]
    augment_count = 0

    print(f"\n📁 Proses kelas: {label} (sudah ada {current_count})")

    while current_count + augment_count < TARGET:
        for file in tqdm(input_files, desc=f"Augment {label}", leave=False):
            if current_count + augment_count >= TARGET:
                break

            filepath = os.path.join(label_input_path, file)
            try:
                data, sr = librosa.load(filepath, sr=None)
            except:
                continue

            aug_type = np.random.choice(['pitch', 'stretch', 'noise', 'shift', 'volume'])

            try:
                if aug_type == 'pitch':
                    augmented = apply_pitch_shift(data, sr)
                elif aug_type == 'stretch':
                    augmented = apply_time_stretch(data)
                elif aug_type == 'noise':
                    augmented = apply_add_noise(data)
                elif aug_type == 'shift':
                    augmented = apply_shift(data)
                elif aug_type == 'volume':
                    augmented = apply_volume_control(data)
                else:
                    continue
            except:
                continue

            new_filename = f"{file[:-4]}_aug{augment_count}_{aug_type}.wav"
            new_filepath = os.path.join(label_output_path, new_filename)
            sf.write(new_filepath, augmented, sr)
            augment_count += 1


📁 Proses kelas: Belly Pain (sudah ada 7)


Augment Belly Pain:   0%|          | 0/279 [00:00<?, ?it/s]


📁 Proses kelas: Burping (sudah ada 0)



📁 Proses kelas: Cold-Hot (sudah ada 0)



📁 Proses kelas: Discomfort (sudah ada 0)



📁 Proses kelas: Hungry (sudah ada 0)



📁 Proses kelas: laugh (sudah ada 0)



📁 Proses kelas: noise (sudah ada 0)



📁 Proses kelas: silence (sudah ada 0)
